<a href="https://colab.research.google.com/github/Alfikriangelo/DeepLearningTasks/blob/main/1stWeekTask/Infrared_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import library
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import math

In [5]:
# Load Dataset
file_path = "sample_data/Infrared.csv"
df = pd.read_csv(file_path)

In [6]:
# Handling Missing Values
df = df.dropna(axis=1)

In [7]:
# One-Hot Encoding
df = pd.get_dummies(df, drop_first=True)

In [8]:
# Memisahkan Features (X) dan Target (y)
X = df.drop(columns=['aveOralM'])
y = df['aveOralM']

In [9]:
# Normalisasi Target (y)
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

In [10]:
# Split Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Normalisasi Features (X)
scaler_x = StandardScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

In [12]:
# MPROVED MODEL - TENSORFLOW
tf_model = keras.Sequential([
    layers.Dense(1024, input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
    layers.Dropout(0.4),
    layers.Dense(512, kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
    layers.Dropout(0.4),
    layers.Dense(256),
    layers.LeakyReLU(),
    layers.Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Compile Model
tf_model.compile(optimizer=keras.optimizers.AdamW(learning_rate=0.0005, weight_decay=1e-5), loss='mse')

In [14]:
# Learning Rate Scheduler
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=20, min_lr=1e-6)

In [16]:
# Training Model
tf_model.fit(X_train_scaled, y_train, epochs=200, batch_size=32, verbose=1, callbacks=[reduce_lr])

Epoch 1/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 1.0862 - learning_rate: 5.0000e-04
Epoch 2/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 1.1465 - learning_rate: 5.0000e-04
Epoch 3/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.1149 - learning_rate: 5.0000e-04
Epoch 4/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 1.0636 - learning_rate: 5.0000e-04
Epoch 5/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.0294 - learning_rate: 5.0000e-04
Epoch 6/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.0889 - learning_rate: 5.0000e-04
Epoch 7/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.0845 - learning_rate: 5.0000e-04
Epoch 8/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.0312 - learning_rate: 5.0000e-04
Epoch 9/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.0157 - learning_rate: 5.0000e-04
Epoch 10/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.0699 - learning_rate: 5.0000e-04
Epoch 11/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/s

In [17]:
# Prediksi TensorFlow
y_pred_tf = tf_model.predict(X_test_scaled)
y_pred_tf = scaler_y.inverse_transform(y_pred_tf)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [18]:
# IMPROVED MODEL - PYTORCH
class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 1)
        self.leaky_relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x

pytorch_model = PyTorchModel()
criterion = nn.MSELoss()
optimizer = optim.AdamW(pytorch_model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [20]:
# Training Model
num_epochs = 200
batch_size = 32
train_data = torch.utils.data.TensorDataset(torch.tensor(X_train_scaled, dtype=torch.float32), torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = pytorch_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(pytorch_model.parameters(), max_norm=1.0)
        optimizer.step()
    scheduler.step()
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [50/200], Loss: 0.0850
Epoch [100/200], Loss: 0.0900
Epoch [150/200], Loss: 0.3712
Epoch [200/200], Loss: 0.1885


In [21]:
# Prediksi PyTorch
y_pred_torch = pytorch_model(torch.tensor(X_test_scaled, dtype=torch.float32)).detach().numpy()
y_pred_torch = scaler_y.inverse_transform(y_pred_torch)

In [22]:
# ============================================
# EVALUASI MODEL
# ============================================
mse_tf = mean_squared_error(scaler_y.inverse_transform(y_test.reshape(-1, 1)), y_pred_tf)
rmse_tf = math.sqrt(mse_tf)
r2_tf = r2_score(scaler_y.inverse_transform(y_test.reshape(-1, 1)), y_pred_tf)

print("\n📌 Evaluasi Model TensorFlow")
print(f"MSE: {mse_tf:.4f}")
print(f"RMSE: {rmse_tf:.4f}")
print(f"R² Score: {r2_tf:.4f}")

mse_torch = mean_squared_error(scaler_y.inverse_transform(y_test.reshape(-1, 1)), y_pred_torch)
rmse_torch = math.sqrt(mse_torch)
r2_torch = r2_score(scaler_y.inverse_transform(y_test.reshape(-1, 1)), y_pred_torch)

print("\n📌 Evaluasi Model PyTorch")
print(f"MSE: {mse_torch:.4f}")
print(f"RMSE: {rmse_torch:.4f}")
print(f"R² Score: {r2_torch:.4f}")


📌 Evaluasi Model TensorFlow
MSE: 0.0612
RMSE: 0.2475
R² Score: 0.7092

📌 Evaluasi Model PyTorch
MSE: 0.0632
RMSE: 0.2513
R² Score: 0.7000


# 📌 **Evaluasi Model TensorFlow & PyTorch**

Evaluasi model dilakukan menggunakan tiga metrik utama: **Mean Squared Error (MSE)**, **Root Mean Squared Error (RMSE)**, dan **R² Score**.

## 1️⃣ **Mean Squared Error (MSE)**
MSE mengukur rata-rata kesalahan kuadrat antara nilai aktual (\(y_i\)) dan nilai prediksi (\(\hat{y}_i\)):

$$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

📊 **Hasil Model:**  
- **TensorFlow:** \(0.0612\)  
- **PyTorch:** \(0.0632\)  

✅ **Interpretasi:**  
Semakin kecil nilai MSE, semakin baik model dalam memprediksi nilai sebenarnya. Model TensorFlow memiliki MSE lebih kecil dibandingkan PyTorch, menunjukkan error yang sedikit lebih rendah.

---

## 2️⃣ **Root Mean Squared Error (RMSE)**
RMSE adalah akar dari MSE, yang memberikan satuan kesalahan yang sama dengan data asli:

$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

📊 **Hasil Model:**  
- **TensorFlow:** \(0.2475\)  
- **PyTorch:** \(0.2513\)  

✅ **Interpretasi:**  
RMSE lebih mudah dipahami karena dalam skala yang sama dengan data asli. Nilai RMSE yang lebih rendah menunjukkan prediksi yang lebih akurat.

---

## 3️⃣ **R² Score (Koefisien Determinasi)**
R² Score mengukur seberapa baik model menjelaskan variasi dalam data, dibandingkan dengan rata-rata sederhana (\(\bar{y}\)):

$$
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$

📊 **Hasil Model:**  
- **TensorFlow:** \(0.7092\)  
- **PyTorch:** \(0.7000\)  

✅ **Interpretasi:**  
R² Score berkisar antara **0 hingga 1**. Semakin mendekati **1**, semakin baik model dalam menjelaskan variasi data. **Model TensorFlow memiliki R² Score yang lebih tinggi dibandingkan PyTorch**, menunjukkan bahwa model TensorFlow lebih baik dalam menangkap pola data.

---

# 🔥 **Kesimpulan & Perbandingan Model**
1. **TensorFlow outperform PyTorch dalam semua metrik**  
   → **MSE lebih kecil, RMSE lebih kecil, dan R² Score lebih tinggi.**
2. **Namun, perbedaan tidak terlalu besar**  
   → **TensorFlow (R² = 0.7092) vs PyTorch (R² = 0.7000)** hanya selisih sekitar **0.01**, yang masih dalam batas wajar.
3. **Potensi Perbaikan:**  
   - Meningkatkan jumlah data  
   - Optimasi lebih lanjut pada hiperparameter  
   - Penyesuaian arsitektur model  

🚀 **Kesimpulan:** Model sudah cukup baik, tapi masih bisa ditingkatkan! 🔥
